Import modules we need

In [ ]:
from utils import *
from gtfs_realtime_utils import *
from google.transit import gtfs_realtime_pb2
ROOT = Path("../")
ROOT.resolve()

We need to unzip the GTFS timetable file to get all the route info. We'll unzip the zip file and store everything in `extract_dir`.

In [ ]:
GTFS_timetable = ROOT / "18SepGB_GTFS_Timetables_Downloaded/itm_yorkshire_gtfs.zip"

# Define the directory where you want to extract files from the timetable
extract_dir = ROOT / "18SepGB_GTFS_Timetables_Downloaded/yorkshire"

unzip_file(GTFS_timetable, extract_dir)

### Dictionaries
To be able to match the live buses to a trip/route etc, we need a way to lookup a given bus's `trip_id` and find its timetabled information. We'll do this using dictionaries because they have [O(1) lookup time thanks to hashmaps](https://dev.to/ajipelumi/how-dictionary-lookup-operations-are-o1-49pk).

We load just the parts we need into pandas dataframes, then use the `to_dict()` method to create our dictionaries for various ID->ID combinations.

We can get more detailed information about UK stops from the [National Public Transport Access Codes data](https://beta-naptan.dft.gov.uk/download). We need the bearing to help match buses to stops on the correct side of the road for their direction of travel.

In [ ]:
stop_names_bearings = get_stop_names_and_bearings()
stop_names_bearings.head()

In [ ]:
# Load the ID parts of various files that we extracted from the GTFS timetable.
agencies, routes, stops, stop_times, trips = load_gtfs_ids(extract_dir)

# Make the dataframes to be used to make dicts
agency2route = agencies.merge(routes, on='agency_id').set_index('agency_id')
AgencyNOC2AgencyIDDict = agency2route['agency_noc'].drop_duplicates().to_dict()

## Routes have multiple associated trips. Need to think about this. 
## Not sure I am using this anywhere so maybe just remove
# route2trip = routes.merge(trips, on='route_id').set_index('route_id')
# RouteID2TripIDDict = route2trip['trip_id'].to_dict()

# Each trip_id has multiple associated stops. Aggregate these into a list per trip_id.
trip2stoptimes = trips.merge(stop_times, on='trip_id')[['trip_id', 'stop_id']].groupby('trip_id').agg(list)
Trip2StopIDDict = trip2stoptimes['stop_id'].to_dict()

# There are some stop_id in stops.txt that are not in stop_times.txt. 
# We can't use these. Our merge removes these as we complete an "inner" join.
stoptimes2stops = stop_times.merge(stops, on='stop_id').merge(stop_names_bearings, how='inner', on='stop_id')
stoptimes2stops.drop_duplicates(subset='stop_id', keep='first', inplace=True) # We only need unique stop_ids
stoptimes2stops.set_index('stop_id', inplace=True)
StopID2StopLocDict = stoptimes2stops[['stop_lat', 'stop_lon', 'Bearing']].to_dict(orient='index')
# print(len(stops.stop_id.unique()))

To check this worked properly, we can see if the length of the dictionary is the same as the number of unique IDs from the original GTFS data.

In [ ]:
assert len(agencies.agency_id.unique()) == len(AgencyNOC2AgencyIDDict)
# assert len(routes.route_id.unique()) == len(RouteID2TripIDDict)
assert len(trips.trip_id.unique()) == len(Trip2StopIDDict)
# assert len(stoptimes2stops.stop_id.unique()) == len(StopID2StopLocDict)

### Get the paths of the GTFS-RT files.

For each of the extracted GTFS-RT files we get their full paths and save it in `gtfs_rt_file_paths`. This is to save some compute time downstream.

In [6]:
# Directory of the GTFS-RT file
sep_15 = 'data/gtfs-rt/extracted_15SepGB_BusLocations_GTFSRT'
sep_18 = 'data/gtfs-rt/extracted_18SepGB_BusLocations_GTFSRT'
sep_19 = 'data/gtfs-rt/extracted_19SepGB_BusLocations_GTFSRT'

gtfs_rt_dir = ROOT / sep_15

# Create an empty list to store file paths
gtfs_rt_file_paths = []

# Walk through the directory
for root, dirs, files in os.walk(gtfs_rt_dir):
    for file in files:
        # Get the full path of the file and append it to the list
        full_path = os.path.abspath(os.path.join(root, file))
        gtfs_rt_file_paths.append(full_path)
        

### Load the feed entities 
We load each feed entity into a list (array in most other languages) to loop through later. We could write this in one big loop, but we've split it up to save compute time and make the code easier to follow/debug.

In [7]:
feed = gtfs_realtime_pb2.FeedMessage()
entities = []
for gtfsrt in gtfs_rt_file_paths:
    # Read the GTFS-RT file
    # Ensure we're reading a binary file and not the readme.md or anything else accidentally.
    if '.bin' in str(gtfsrt):
        with open(gtfsrt, 'rb') as file:
            feed.ParseFromString(file.read())
            for entity in feed.entity:
                entities.append(entity)

### Getting the real-time locations of Buses

Now we want to iterate through each minute of the GTFS-RT feed data.
- For each feed message, we iterate through every entity. 
- A single entity contains information about a single trip (bus). 
- Per entity, we instantiate a `Class` called `BusDetail` and save the current trip info to that class.
- If the trip_id exists, we find all the stops on that route using the trip_id.
- Get the locations of the stops on that route.
- Calculate a bounding box...
- Check the bearing using... 
- Calculate the distance to each stop in the simplified list of stops using the Haversine formula.
- Get the `stop_id` and distance of the nearest stop (smallest distance)
- If the nearest stop exists and is closer than 200m, we add this bus to our bag, `BusDetailsBag` for later.

In [ ]:
BusDetailsBag = []
count = 0
no_trip_id = 0
t0 = time.time()
# For the moment assuming that `vehicle` is passed. there are other mutually exclusive 
# options: 'trip_update', 'alert', and 'shape'. See the DOCS https://gtfs.org/documentation/realtime/reference/#message-feedentity
for entity in entities:
    BD = BusDetail()
    # These are the two main parts of entity
    vehicle = entity.vehicle
    # # These are sub parts
    trip = vehicle.trip
    pos = vehicle.position
    # These are individual values
    BD.feed_uid = entity.id
    BD.trip_id = trip.trip_id
    BD.route_id = trip.route_id
    BD.lat = round(pos.latitude, 6)
    BD.lon = round(pos.longitude, 6)
    BD.bearing = pos.bearing
    BD.ts = vehicle.timestamp
    BD.v_id = vehicle.vehicle.id
    BD.occupancy_status = vehicle.occupancy_status
    BD.current_stop_sequence = vehicle.current_stop_sequence
    BD.current_status = vehicle.current_status

    if BD.trip_id:
        stops_on_route = Trip2StopIDDict.get(BD.trip_id)
        # If stops isn't none - i.e. if this trip ID is part of the WY timetable
        if stops_on_route:
            # Get the stop_id, lat, and lon for each stop on the route.
            # Scaling the longitudes because they get closer together nearer the poles. Could replace BD.lat with cos(~53) for UK average
            actual_stop_locations_on_route = [
                (stopid, StopID2StopLocDict[stopid]['stop_lat'], 
                 StopID2StopLocDict[stopid]['stop_lon'], 
                 abs(StopID2StopLocDict[stopid]['Bearing'] - BD.bearing), 
                 abs(StopID2StopLocDict[stopid]['stop_lat'] - BD.lat), 
                 abs((StopID2StopLocDict[stopid]['stop_lon'] - BD.lon)/np.cos(BD.lat))
                ) for stopid in stops_on_route
            ]
            
            box_size = 0.003 # 0.01 is ~1.1km 
            # @TODO ADD BUS BEARING HERE. Can filter out all stops that are pointing the "wrong direction". Absolute difference between stop bearing and bus bearing should be < 90 degrees. This gives a semi-circle's worth of error margin.
            
            stops_in_bounds = [item for item in actual_stop_locations_on_route if (item[4] < box_size) and (item[5] < box_size) and (item[3] < 90) and (item[3] != 'nan')]
            
            if len(stops_in_bounds) > 0:
                candidate_stops_and_distances = [item + (haversine(BD.lat, BD.lon, item[1], item[2]),) for item in stops_in_bounds]
    
                n_possible_stops = len(candidate_stops_and_distances)
                if n_possible_stops == 1:
                    # Found the nearest stop already. Get the distance and stop_id.
                    closest_stop_id = candidate_stops_and_distances[0][0]
                    closest_stop_distance = candidate_stops_and_distances[0][6]
                else:
                    # Need to get min distance from n stops.
                    index_of_smallest_distance = min(range(len(candidate_stops_and_distances)), key=lambda i: candidate_stops_and_distances[i][6])
                    closest_stop_id = candidate_stops_and_distances[index_of_smallest_distance][0]
                    closest_stop_distance = candidate_stops_and_distances[index_of_smallest_distance][6]
                
                # Add the details to the current bus.
                BD.NearestStopOnRoute = closest_stop_id
                BD.NearestStopDistance = closest_stop_distance * 1e3 #convert to m

                # Ensure we found a nearest stop and that the bus is "sufficiently" close.
                if BD.NearestStopOnRoute != None and BD.NearestStopDistance < 200:
                    BusDetailsBag.append(BD)
        else:
            no_trip_id += 1
    # Creating some info to see progress
    if count!= 0 and count % 500000 == 0:
        t1 = time.time()
        print('Time elapsed:', round(t1-t0, 3), 's')
        print(f"{count} of {len(entities)} entities parsed.")
    count +=1
print(f"Number of buses with no trip_id: {no_trip_id}")

Now delete duplicate stops (this also gets rid of long periods where the vehicle is waiting)

Sort the frame by timestamp descending, then keep FIRST of duplicate rows.

In [9]:
data = [{'trip_id': bus.trip_id, "route_id": bus.route_id, 'timestamp': bus.ts, 'nearest_stop_id': bus.NearestStopOnRoute, 'distance': bus.NearestStopDistance} for bus in BusDetailsBag]

# Add the data to a dataframe
df = pd.DataFrame(data)

# Work out the human readable time
df['human_time'] = pd.to_datetime(df['timestamp'], unit='s')

# Timezone is currently BST =  UTC + 1, so need to add 1 hour.
df['uk_bst_time_only'] = (df['human_time'] + pd.Timedelta(hours=1)).dt.strftime('%H:%M:%S')

# Weird rows where they aren't the right date. Binning them
df = df.loc[df.human_time.dt.date == pd.to_datetime('2024-09-15').date()]

# only remove duplictaes that have same stop_id, route_id and nearest_stop_id
df.drop_duplicates(subset=['trip_id', 'route_id', 'nearest_stop_id'], keep='first', inplace=True)

# # For a given trip_id, the arrival_time of (n+1)-th stoptime in sequence must not precede the departure_time of n-th stoptime in sequence in stop_times.txt.
# df.sort_values(by=['timestamp', 'trip_id', 'route_id'], ascending=True, inplace=True)

FilteredOrderedBusLocations = df.copy()
# FilteredOrderedBusLocations[FilteredOrderedBusLocations.trip_id == 'VJ79ee6abdf8b1015398ec41a7f77b76c7f6a0b10f']

Reading in the original GTFS timetable data. We'll use this to pull all the info about the routes based on the trips we were actually able to match buses for.

In [10]:
agencies, routes, trips, stops, stop_times, calendar, calendar_dates, feed_info, shapes = load_full_gtfs(extract_dir, include=['feed_info.txt', 'shapes.txt'])

Now write this timetable as GTFS. This includes:
- agency.txt
- calendar.txt
- calendar_dates.txt
- routes.txt
- stop_times.txt
- stops.txt 
- trips.txt
- shapes.txt

In [11]:
war_datadir = ROOT / "war/yorkshire/150924/"
os.makedirs(os.path.abspath(war_datadir), exist_ok=True)

In [ ]:
RealRouteIDs = FilteredOrderedBusLocations['route_id'].astype(int)
RealRoutes = routes[routes['route_id'].isin(RealRouteIDs)]
RealRoutes.to_csv(war_datadir / "routes.txt", index=False)
RealRoutes

In [ ]:
RealAgencyIDs = routes['agency_id']
RealAgencies = agencies[agencies['agency_id'].isin(RealAgencyIDs)]
RealAgencies.to_csv(war_datadir / "agency.txt", index=False)
RealAgencies

In [ ]:
RealTripIDs = FilteredOrderedBusLocations['trip_id']
RealTrips = trips[trips['trip_id'].isin(RealTripIDs)]
RealTrips.to_csv(war_datadir / "trips.txt", index=False)
RealTrips

In [15]:
RealShapeIDs = RealTrips['shape_id'].unique()
RealShapes = shapes[shapes.shape_id.isin(RealShapeIDs)]
RealShapes.to_csv(war_datadir / "shapes.txt", index=False)

In [16]:
RealServiceIDs = RealTrips['service_id'].unique()
RealCalendar = calendar[calendar['service_id'].isin(RealServiceIDs)]
RealCalendarDates = calendar_dates[calendar_dates['service_id'].isin(RealServiceIDs)]
RealCalendar.to_csv(war_datadir / "calendar.txt", index=False)
RealCalendarDates.to_csv(war_datadir / "calendar_dates.txt", index=False)

Stop times.txt.

minimally required fields are trip_id, arrival_time and/or departure_time, stop_id, and stop_sequence (must be sequential)

In [17]:
RealTripIDs_list = RealTripIDs.to_list()
FilteredStopTimes = stop_times[stop_times.trip_id.isin(RealTripIDs_list)]
FilteredOrderedBusLocations.rename(columns={'nearest_stop_id': 'stop_id'}, inplace=True)
RealTrips2RealStopsDict = FilteredStopTimes.groupby('trip_id')['stop_id'].agg(list).to_dict()

In [18]:
RealStopTimes = FilteredStopTimes.merge(FilteredOrderedBusLocations, on=['trip_id', 'stop_id'], how='inner')
RealStopTimes['arrival_time'] = RealStopTimes['uk_bst_time_only']
RealStopTimes['departure_time'] = RealStopTimes['uk_bst_time_only']

# A trip must visit more than one stop in stop_times.txt to be usable by passengers for boarding and alighting.
stop_counts = RealStopTimes.groupby('trip_id')['stop_id'].count()
trip_ids_with_one_stop = stop_counts[stop_counts == 1].index.to_list()
RealStopTimes = RealStopTimes[~RealStopTimes.trip_id.isin(trip_ids_with_one_stop)] # Exclude trips with only 1 stop. (The "~" is negation)

# When sorted by stop_times.stop_sequence, two consecutive entries in stop_times.txt 
# should have increasing distance, based on the field shape_dist_traveled. 
# If the values are equal, this is considered as an error.
RealStopTimes.sort_values(by=['trip_id', 'stop_sequence'], ascending=True, inplace=True)

# If pick up type == 1, no pickup is available (Guessing this means you can't get on the bus here?)
RealStopTimes = RealStopTimes[RealStopTimes.pickup_type != 1]

# Remove duplicate rows based on the specified columns but keep the first occurrence
RealStopTimes = RealStopTimes.drop_duplicates(subset=['trip_id', 'stop_id', 'arrival_time', 'departure_time', 'shape_dist_traveled'], keep='first')

# Filtering only columns we need to write stop_times.txt
RealStopTimes = RealStopTimes[['trip_id','arrival_time','departure_time','stop_id','stop_sequence','stop_headsign','pickup_type','drop_off_type','shape_dist_traveled','timepoint']]
# RealStopTimes[RealStopTimes.trip_id == "VJ79ee6abdf8b1015398ec41a7f77b76c7f6a0b10f"]

RealStopTimes.to_csv(war_datadir / "stop_times.txt",index=False)
# RealStopTimes[RealStopTimes.trip_id == 'VJ8c815c88e8f60f0f1d3f161565a670bf5de81ed6']

Stops.txt

In [19]:
RealStopIDs = FilteredOrderedBusLocations['stop_id'].to_list()
RealStops = stops[stops.stop_id.isin(RealStopIDs)].copy()

# Location type must be able to be parsed as an integer.
RealStops['location_type'] = RealStops['location_type'].astype('Int64')
RealStops.drop(columns='parent_station', inplace=True)
RealStops.to_csv(war_datadir / "stops.txt", index=False)

### Creating and writing `feed_info.txt`

In [20]:
feed_info.to_csv(war_datadir / "feed_info.txt", index=False)

Need to write some code to automaticall zip these files.

In [21]:
import shutil
def zip_directory(folder_path, output_dir_path, output_filename):
    # Ensure the output filename does not have a .zip extension
    if not output_filename.endswith('.zip'):
        output_filename += '.zip'
    # Join the output directory and filename
    output_zip_path = output_dir_path / output_filename
    
    # Create the zip archive in the specified directory
    shutil.make_archive(output_zip_path.with_suffix(''), 'zip', folder_path)
    print(f"Directory '{folder_path}' successfully zipped as '{output_filename}'.")

In [ ]:
zip_directory(war_datadir, ROOT / 'war', 'yorkshire_150924.gtfs')